In [95]:
from functools import reduce
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import pandas as pd

# API 키 설정
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 언어 모델 초기화
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0)

# 텍스트 분할 설정
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=100,
)

# 문서 로드 및 분할
file_path = "../../data/job/all.csv"

# 각 파일을 로드하고 분할하여 문서 리스트를 반환
def load_and_split(file_path):
    loader = CSVLoader(file_path, encoding='utf8')
    return loader.load_and_split(text_splitter=splitter)

# 문서 로드 및 분할
docs = load_and_split(file_path)

# 임베딩과 캐싱 설정
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터 저장소 생성
vectorstore = FAISS.from_documents(docs, embeddings)

In [96]:
# save db
vectorstore.save_local("../../models/gpt/db")

# load db
db = FAISS.load_local("../../models/gpt/db", embeddings, allow_dangerous_deserialization=True)

# 검색기 설정
retriever = db.as_retriever(search_kwargs={'k': 5})

In [97]:
# 프롬프트 설정
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            당신은 직업 추천 컨설턴트입니다. 
            당신은 저장된 이력서를 활용하여 사용자의 이력서를 분석하고, 직업 추천/추천이유/보완점을 출력 포맷에 맞춰 작성해 주셔야 합니다. 
            
            추천 절차는 아래와 같습니다. 
            1. 사용자의 이력서와 저장된 이력서 파일과의 유사성을 평가합니다.
            2. 저장된 이력서 데이터에서 유사성이 높은 상위 2개의 직업을 저장합니다. 
            3. 유사성이 높은 순서대로 출력 포맷에 맞춰 직업, 추천 이유, 보완점을 출력합니다.
            4. 추천 이유: 해당 직업을 추천하는 구체적인 이유를 제공합니다.
            5. 보완점: 해당 직업에 더 적합해지기 위해 보완해야 할 주요 스킬이나 경험을 제안합니다.
            6. 각 추천 직업에 대해 다음과 같은 형식으로 정보 제공:

            1. 직업명
            - 추천 이유: 해당 직업을 추천하는 구체적인 이유
            - 보완점: 해당 직업에 더 적합해지기 위해 이력서에서 보완해야 할 주요 스킬이나 경험을 제안           

            
            아래는 잘 작성된 몇 가지 이력서 첨삭 예시입니다.
            아래 예시들을 **참고만 하고**, 저장된 이력서 데이터와 비교했을때 사용자가 보완해야할 점을 최대한 반영하여 이력서를 첨삭해주세요. 

            1. 데이터 엔지니어
            - 추천 이유: Python과 SQL을 활용한 데이터 처리 능력이 있습니다. 데이터베이스 설계와 관리에 대한 이해를 바탕으로 데이터 인프라를 구축 및 최적화하는 역량이 있습니다.
            - 보완점: Python, Java, Scala 등 주요 코딩 스킬을 향상시키고, Spark, Hadoop 같은 분산처리 시스템 아키텍처에 대한 이해를 높여야 합니다. REST API 개발, AWS 인프라 운영과 ETL 파이프라인 구축 경험을 통해 실시간 대용량 데이터 처리 경험을 쌓는 것이 중요합니다. 

            2. 백엔드 개발자
            - 추천 이유: Java, Python 등의 프로그래밍 언어 능력과 MySQL, Oracle 등의 DBMS 이해 능력이 있습니다. 복잡한 백엔드 시스템의 아키텍처 설계 및 개발 경험을 갖추고 있습니다. 
            - 보완점: Spring Boot, Django와 같은 웹 프레임워크를 통한 RESTful API 개발 경험을 확대하고, AWS 클라우드 서비스와 Docker, Kubernetes를 사용한 컨테이너 기반 배포와 오케스트레이션 능력을 개발하는 것이 중요합니다. 또한, 마이크로서비스 아키텍처에 대한 이해를 키워야 합니다.

            """
        ),
        ("human", "{question}")
    ]
)


# 실행 체인 정의 및 결과 출력
chain = (
    {
        "context": retriever,
        "question":RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

question = """
    "활용 스킬": "Python, R, SQL, Excel",
    "수상 내역": "환경 데이터 분석 경진대회에서 최우수상을 수상했습니다.",
    "동아리": "교내 데이터 분석 동아리에서 활동하며 다양한 데이터 분석 프로젝트를 진행했습니다.",
    "프로젝트": "온라인 소매 데이터를 분석하여 고객 세분화 및 타겟 마케팅 전략을 개발했습니다. 주요 사용 기술은 Python, SQL, 그리고 R이며, 데이터 전처리부터 통계적 분석, 예측 모델링까지 전 과정을 담당했습니다."
"""
result = chain.invoke(question)
print(result)

저장된 이력서 데이터와 사용자의 이력서를 비교한 결과, 사용자의 프로필과 가장 일치하는 직업은 데이터 엔지니어와 데이터 사이언티스트입니다. 아래는 각 직업에 대한 추천 이유와 보완점을 안내해드리겠습니다.

### 1. 데이터 엔지니어
- 추천 이유: Python과 SQL을 활용한 데이터 처리 능력이 있으며, 데이터베이스 설계와 관리에 대한 이해를 바탕으로 데이터 인프라를 구축하고 최적화하는 역량이 있습니다.
- 보완점: 주요 코딩 스킬인 Python, Java, Scala 등을 더 향상시키고, 분산처리 시스템 아키텍처에 대한 이해를 높이는 것이 필요합니다. 또한, REST API 개발, AWS 인프라 운영, ETL 파이프라인 구축 경험을 통해 실시간 대용량 데이터 처리 경험을 쌓는 것이 중요합니다.

### 2. 데이터 사이언티스트
- 추천 이유: Python, R, SQL 등 다양한 데이터 분석 도구와 언어를 활용한 경험이 있으며, 환경 데이터 분석 경진대회에서 최우수상을 수상한 경력이 있습니다.
- 보완점: 머신러닝 및 딥러닝 알고리즘에 대한 이해와 적용 경험을 더 쌓는 것이 중요합니다. 또한, 대용량 데이터 처리를 위한 분산 시스템 및 클라우드 플랫폼(AWS, GCP) 경험을 키우고, 데이터 시각화 및 커뮤니케이션 능력을 강화하는 것이 도움이 될 것입니다.

이러한 보완점을 고려하여 자신의 경험과 스킬을 더 발전시키면 데이터 엔지니어 또는 데이터 사이언티스트로의 경력을 더욱 확실하게 쌓을 수 있을 것입니다.


In [98]:
# 데이터 프레임으로 변환
sections = result.split('### ')[1:] 
data = {
    "직업명": [],
    "추천 이유": [],
    "보완점": []
}

for section in sections:
    lines = section.strip().split('\n')
    job_name = lines[0].split('. ', 1)[1]
    recommendation_reason = lines[1].split(': ', 1)[1]
    improvement_points = lines[2].split(': ', 1)[1]
    data["직업명"].append(job_name)
    data["추천 이유"].append(recommendation_reason)
    data["보완점"].append(improvement_points)

df = pd.DataFrame(data)
df 

,직업명,추천 이유,보완점
0,데이터 엔지니어,"Python과 SQL을 활용한 데이터 처리 능력이 있으며, 데이터베이스 설계와 관리...","주요 코딩 스킬인 Python, Java, Scala 등을 더 향상시키고, 분산처리..."
1,데이터 사이언티스트,"Python, R, SQL 등 다양한 데이터 분석 도구와 언어를 활용한 경험이 있으...",머신러닝 및 딥러닝 알고리즘에 대한 이해와 적용 경험을 더 쌓는 것이 중요합니다. ...
